# Prédiction des tags

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

datas = pd.read_csv("../P5_06_transf_datas.csv")

datas

,stem_words,imp_words,Tags
0,convert decimal double c# want use track bar c...,convert decimal double c# want track bar chang...,c# floating-point type-conversion double ...
1,width collapse percentage width child element ...,width collapse percentage width child element ...,html css internet-explorer-7
2,calculate someone age base datetime type birth...,calculate someone age base datetime type give ...,c# .net datetime
3,calculate relative time c# give specific datet...,calculate relative time c# give specific datet...,c# datetime time datediff relative-time-s...
4,determine user timezone standards way web serv...,determine user timezone standards way web serv...,html browser timezone user-agent timezone...
...,...,...,...
29096,extract decimal part float point number extrac...,extract decimal part float point number extrac...,c floating-point decimal numbers
29097,flash load twice workaround investigate report...,flash load twice workaround investigate report...,flash swfobject
29098,etag vs header expire look around able figure ...,vs header expire look around able figure expir...,http caching etag expires-header
29099,determine longest similar portion several stri...,determine similar portion several string per t...,perl algorithm string similarity


Je sépare le jeu de données en un jeu d'entraînement et un de test.

In [2]:
X_train = datas["imp_words"]
Y_train = datas["Tags"]

In [3]:
# samples = np.random.choice(len(X_train), 5000, replace=False)
# X_train = X_train[samples]
# Y_train = Y_train[samples]

Je  transforme X et y.

In [4]:
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer

class TfidfVectorsToArray(TransformerMixin):
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.toarray()

# class YToBagOfWords(CountVectorizer):
#     def fit(self, X= None, y= y_train, **fit_params):
#         return super().fit(y)

#     def transform(self, X= None, y= y_train, **fit_params):
#         return super().transform(y)

#     def fit_transform(self, X= None, y= y_train, **fit_params):
#         return super().fit_transform(y)

# class YToArray(TransformerMixin):
#     def fit(self, X= None, y= y_train, **fit_params):
#         return self

#     def transform(self, X= None, y= y_train, **fit_params):
#         return y.toarray()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[
    ("tfidf", TfidfVectorizer()),
    ("toarray", TfidfVectorsToArray()),
    ("PCA", PCA(n_components= 1500)),
    ("clf", OneVsRestClassifier(
        SGDClassifier(loss= "modified_huber", class_weight= None, 
        penalty= "l1", alpha= 0.0001, l1_ratio= 0.15, early_stopping= True, 
        random_state= 42)))
    ])

# tfidf_vect = TfidfVectorizer(token_pattern= r'[\S]+')

# x_train = tfidf_vect.fit_transform(X_train).toarray()


In [6]:
# component = 1500

# pca = PCA(n_components= component)
# pca.fit(x_train)

# pca.explained_variance_ratio_.cumsum()[-1]

In [7]:
# x_train_pca = pca.transform(x_train)


In [8]:
count_vect = CountVectorizer(token_pattern= r'[\S]+', min_df= 10)
y_train = count_vect.fit_transform(Y_train).toarray()


In [9]:
with open("tags.txt", "w") as file:
    file.write(" ".join(count_vect.get_feature_names_out()))

In [10]:
# y_train.shape

'score': 0.9919314591700134,
 'loss': 'modified_huber',
 'class_weight': None,
 'penalty': 'l1',
 'alpha': 0.0001,
 'l1_ratio': 0.15

In [11]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import SGDClassifier

# clf = OneVsRestClassifier(SGDClassifier(loss= "modified_huber", class_weight= None, 
#     penalty= "l1", alpha= 0.0001, l1_ratio= 0.15, early_stopping= True, 
#     random_state= 42))

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('toarray',
                 <__main__.TfidfVectorsToArray object at 0x0000021C4AFD0E80>),
                ('PCA', PCA(n_components=1500)),
                ('clf',
                 OneVsRestClassifier(estimator=SGDClassifier(early_stopping=True,
                                                             loss='modified_huber',
                                                             penalty='l1',
                                                             random_state=42)))])

In [19]:
pred = pipeline.predict_proba(["c# don't open mysql database with windows access file "])

In [13]:
pred.shape

(1, 1068)

In [20]:
results_preds = np.zeros(pred.shape)

pred_sorted = np.sort(pred, axis= 1)
diff = pred_sorted[:, -1] - np.log10(pred_sorted[:, -1] +1)

results_preds = np.where(pred > diff.reshape(-1, 1), 1, 0)

In [15]:
results_preds.shape

(1, 1068)

In [16]:
with open("tags.txt", "r") as file:
    cols = file.read().split()


In [21]:
# cols = count_vect.get_feature_names_out()

results_df = pd.DataFrame(results_preds, columns= cols).T.reset_index()
" ".join(results_df.loc[results_df[0] > 0, "index"].values)

'mysql'

Sauvegarde du modèle

In [23]:
# import joblib

# joblib.dump(pipeline, "P5_tags_prediction.sav")

['P5_tags_prediction.sav']